# Guide for creating machine learning model to predict if credit card shall be approved or denied 

    import libraries that we will use
   

In [79]:
! pip3 install pandas
! pip3 install numpy
! pip3 install sklearn
! pip3 install matplotlib
! pip3 install scipy
! pip3 install seaborn

In [80]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [81]:
df = pd.read_csv("data.csv")

In [82]:
print (df)

     YearsEmployed  PriorDefault  Credi1Score  Income  approved
0            1.250             1            1       0         1
1            3.040             1            6     560         1
2            1.500             1            0     824         1
3            3.750             1            5       3         1
4            1.710             1            0       0         1
5            2.500             1            0       0         1
6            6.500             1            0   31285         1
7            0.040             1            0    1349         1
8            3.960             1            0     314         1
9            3.165             1            0    1442         1
10           2.165             0            0       0         1
11           4.335             1            0     200         1
12           1.000             1            0       0         1
13           0.040             0            0    2690         1
14           5.000             1        

In [83]:
filt_df = df.loc[:, df.columns != 'PriorDefault']
filt_df = filt_df.loc[:, filt_df.columns != 'Credi1Score']
filt_df = filt_df.loc[:, filt_df.columns != 'approved']
print (filt_df)


     YearsEmployed  Income
0            1.250       0
1            3.040     560
2            1.500     824
3            3.750       3
4            1.710       0
5            2.500       0
6            6.500   31285
7            0.040    1349
8            3.960     314
9            3.165    1442
10           2.165       0
11           4.335     200
12           1.000       0
13           0.040    2690
14           5.000       0
15           0.250       0
16           0.960       0
17           3.170     245
18           0.665       0
19           0.750       0
20           2.500    1208
21           0.835       0
22           7.875    1260
23           3.085      11
24           0.500       0
25           1.500       0
26           5.165       0
27          15.000       0
28           7.000       0
29           5.000   10000
..             ...     ...
645          0.085       0
646          0.165       0
647          0.875       0
648          1.500       0
649          0.040       0
6

In [84]:
low = .05
high = .95
quant_df = filt_df.quantile([low, high])
print(quant_df)

      YearsEmployed  Income
0.05         0.0000     0.0
0.95         8.5375  4021.3


In [85]:
filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)
# filt_df = pd.concat([filt_df, df['approved']], axis=1)
print (filt_df)


     YearsEmployed  Income
0            1.250     NaN
1            3.040   560.0
2            1.500   824.0
3            3.750     3.0
4            1.710     NaN
5            2.500     NaN
6            6.500     NaN
7            0.040  1349.0
8            3.960   314.0
9            3.165  1442.0
10           2.165     NaN
11           4.335   200.0
12           1.000     NaN
13           0.040  2690.0
14           5.000     NaN
15           0.250     NaN
16           0.960     NaN
17           3.170   245.0
18           0.665     NaN
19           0.750     NaN
20           2.500  1208.0
21           0.835     NaN
22           7.875  1260.0
23           3.085    11.0
24           0.500     NaN
25           1.500     NaN
26           5.165     NaN
28           7.000     NaN
29           5.000     NaN
30           5.040     NaN
..             ...     ...
644          0.250   154.0
645          0.085     NaN
646          0.165     NaN
647          0.875     NaN
648          1.500     NaN
6

In [86]:
print (df)

     YearsEmployed  PriorDefault  Credi1Score  Income  approved
0            1.250             1            1       0         1
1            3.040             1            6     560         1
2            1.500             1            0     824         1
3            3.750             1            5       3         1
4            1.710             1            0       0         1
5            2.500             1            0       0         1
6            6.500             1            0   31285         1
7            0.040             1            0    1349         1
8            3.960             1            0     314         1
9            3.165             1            0    1442         1
10           2.165             0            0       0         1
11           4.335             1            0     200         1
12           1.000             1            0       0         1
13           0.040             0            0    2690         1
14           5.000             1        

In [87]:
filt_df = pd.concat([df.loc[:,'PriorDefault'],df.loc[:,'Credi1Score'],df.loc[:,'approved'], filt_df], axis=1)

In [88]:
print (filt_df)

     PriorDefault  Credi1Score  approved  YearsEmployed  Income
0               1            1         1          1.250     NaN
1               1            6         1          3.040   560.0
2               1            0         1          1.500   824.0
3               1            5         1          3.750     3.0
4               1            0         1          1.710     NaN
5               1            0         1          2.500     NaN
6               1            0         1          6.500     NaN
7               1            0         1          0.040  1349.0
8               1            0         1          3.960   314.0
9               1            0         1          3.165  1442.0
10              0            0         1          2.165     NaN
11              1            0         1          4.335   200.0
12              1            0         1          1.000     NaN
13              0            0         1          0.040  2690.0
14              1            7         1

In [89]:
filt_df.dropna(inplace=True)
print (filt_df)

     PriorDefault  Credi1Score  approved  YearsEmployed  Income
1               1            6         1          3.040   560.0
2               1            0         1          1.500   824.0
3               1            5         1          3.750     3.0
7               1            0         1          0.040  1349.0
8               1            0         1          3.960   314.0
9               1            0         1          3.165  1442.0
11              1            0         1          4.335   200.0
13              0            0         1          0.040  2690.0
17              1           10         1          3.170   245.0
20              1           17         1          2.500  1208.0
22              1            6         1          7.875  1260.0
23              1            1         1          3.085    11.0
33              1            0         1          5.000  4000.0
34              1            5         1          0.415   560.0
35              1           11         1